In [1]:
import numpy as np
import pandas as pd 

from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import rdMolDescriptors
import rdkit.Chem as Chem

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, KFold

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

#from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from mordred import Calculator, descriptors
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from imblearn.over_sampling import SMOTE



In [2]:
test_data = pd.read_csv('test_II.csv')
train_data = pd.read_csv('train_II.csv')
desc_df = pd.read_csv('descriptorsfromrdkitv1.csv')

In [3]:
test_data.head()

,x
0,CC1=CC(=C(C=C1)C(C)(C)C)O;1682
1,CCS(=O)(=O)C1=C(N=CC=C1)S(=O)(=O)NC(=O)NC2=NC(...
2,CC1=NC2=CC=CC=C2N1C3CC4CCC(C3)N4CCC5(CCN(CC5)C...
3,CC1=CC(=O)[N-]S(=O)(=O)O1.[K+];1850
4,CCC(C)(C)C(=O)O[C@H]1C[C@H](C=C2[C@H]1[C@H]([C...


In [4]:
train_data[['Chemical ID', 'Assay ID']] = train_data['Id'].str.split(';', expand=True)

In [5]:
train_data = train_data.drop('Id', axis=1)

In [6]:
train_data.head()


,Expected,Chemical ID,Assay ID
0,2,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O,1644
1,2,CCCCCCCCC(=O)C,2451
2,2,CCCCCCCCCC[N+](C)(C)CCCCCCCCCC.[Cl-],1384
3,2,C1CN(C(=N1)N[N+](=O)[O-])CC2=CN=C(C=C2)Cl,16
4,2,[Na+].[I-],1856


In [7]:
#trd_chem_aid_df.head()

In [8]:
train_data = train_data.rename(columns={'Expected': 'Label'})


In [9]:
train_data.head()

,Label,Chemical ID,Assay ID
0,2,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O,1644
1,2,CCCCCCCCC(=O)C,2451
2,2,CCCCCCCCCC[N+](C)(C)CCCCCCCCCC.[Cl-],1384
3,2,C1CN(C(=N1)N[N+](=O)[O-])CC2=CN=C(C=C2)Cl,16
4,2,[Na+].[I-],1856


In [10]:
# Create a list to store the SMILES strings that do not cause an error
valid_smiles = []

# Iterate over the "Chemical ID" column and store only the valid SMILES strings
for smiles in train_data["Chemical ID"]:
    try:
        mols = Chem.MolFromSmiles(smiles, sanitize=False)
        if mols is not None:
            valid_smiles.append(smiles)
    except:
        continue

# Filter the DataFrame to only include rows with valid SMILES strings
train_data = train_data[train_data["Chemical ID"].isin(valid_smiles)]


In [11]:
trd_chem_aid_df = train_data[['Chemical ID', 'Assay ID']]

In [12]:
#Method transforms smiles strings to mol rdkit object
#train_data['mols'] = train_data['Chemical ID'].apply(lambda x: Chem.MolFromSmiles(x)) 



#train_data = train_data.dropna(subset=['mols'])
train_data.head()

,Label,Chemical ID,Assay ID
0,2,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O,1644
1,2,CCCCCCCCC(=O)C,2451
2,2,CCCCCCCCCC[N+](C)(C)CCCCCCCCCC.[Cl-],1384
3,2,C1CN(C(=N1)N[N+](=O)[O-])CC2=CN=C(C=C2)Cl,16
4,2,[Na+].[I-],1856


In [13]:
print(train_data.isnull().sum())

Label          0
Chemical ID    0
Assay ID       0
dtype: int64


In [14]:
train_data['mols'] = train_data['Chemical ID'].apply(lambda x: Chem.MolFromSmiles(x))

[22:00:02] Explicit valence for atom # 1 Si, 8, is greater than permitted
[22:00:04] Explicit valence for atom # 1 Si, 8, is greater than permitted
[22:00:05] Explicit valence for atom # 1 Si, 8, is greater than permitted
[22:00:06] Explicit valence for atom # 1 Si, 8, is greater than permitted
[22:00:08] Explicit valence for atom # 1 Si, 8, is greater than permitted
[22:00:08] Explicit valence for atom # 1 Si, 8, is greater than permitted


In [15]:
print(train_data.isnull().sum())

Label          0
Chemical ID    0
Assay ID       0
mols           6
dtype: int64


In [16]:
train_data.head()

,Label,Chemical ID,Assay ID,mols
0,2,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O,1644,<rdkit.Chem.rdchem.Mol object at 0x000002363FC...
1,2,CCCCCCCCC(=O)C,2451,<rdkit.Chem.rdchem.Mol object at 0x000002363FC...
2,2,CCCCCCCCCC[N+](C)(C)CCCCCCCCCC.[Cl-],1384,<rdkit.Chem.rdchem.Mol object at 0x000002363FC...
3,2,C1CN(C(=N1)N[N+](=O)[O-])CC2=CN=C(C=C2)Cl,16,<rdkit.Chem.rdchem.Mol object at 0x000002363FC...
4,2,[Na+].[I-],1856,<rdkit.Chem.rdchem.Mol object at 0x000002363FC...


In [17]:
# Calculate the molecular descriptors for each molecule
#descs = []
#for i, mol in enumerate(train_data['mols']):
 #   desc = []
  #  for d in Descriptors.descList:
   #     try:
    #        desc.append(d[1](mol))
     #   except:
      #      desc.append(np.nan)
   # descs.append(desc)

In [18]:
train_data = train_data.dropna(subset=['mols'])
#DECIDE WHAT TO DO WITH MISSING VALUES

In [19]:
print(train_data.isnull().sum())

Label          0
Chemical ID    0
Assay ID       0
mols           0
dtype: int64


In [20]:
# Generate the Morgan fingerprints from the RDkit objects
#train_data['morgan_fp'] = train_data['mols'].apply(lambda x: rdMolDescriptors.GetMorganFingerprintAsBitVect(x, radius =4 , nBits = 300, useFeatures=True, useChirality=True))


In [21]:
train_data.head()

,Label,Chemical ID,Assay ID,mols
0,2,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O,1644,<rdkit.Chem.rdchem.Mol object at 0x000002363FC...
1,2,CCCCCCCCC(=O)C,2451,<rdkit.Chem.rdchem.Mol object at 0x000002363FC...
2,2,CCCCCCCCCC[N+](C)(C)CCCCCCCCCC.[Cl-],1384,<rdkit.Chem.rdchem.Mol object at 0x000002363FC...
3,2,C1CN(C(=N1)N[N+](=O)[O-])CC2=CN=C(C=C2)Cl,16,<rdkit.Chem.rdchem.Mol object at 0x000002363FC...
4,2,[Na+].[I-],1856,<rdkit.Chem.rdchem.Mol object at 0x000002363FC...


In [22]:
train_data.shape[0]

75377

In [23]:
train_data.head()

,Label,Chemical ID,Assay ID,mols
0,2,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O,1644,<rdkit.Chem.rdchem.Mol object at 0x000002363FC...
1,2,CCCCCCCCC(=O)C,2451,<rdkit.Chem.rdchem.Mol object at 0x000002363FC...
2,2,CCCCCCCCCC[N+](C)(C)CCCCCCCCCC.[Cl-],1384,<rdkit.Chem.rdchem.Mol object at 0x000002363FC...
3,2,C1CN(C(=N1)N[N+](=O)[O-])CC2=CN=C(C=C2)Cl,16,<rdkit.Chem.rdchem.Mol object at 0x000002363FC...
4,2,[Na+].[I-],1856,<rdkit.Chem.rdchem.Mol object at 0x000002363FC...


In [24]:
# Get the fingerprints and labels from the dataframe
#X_train = np.array(train_data['morgan_fp'].tolist())
#assay_id = np.array(train_data['Assay ID'].values).reshape(-1, 1)
#y_train = train_data['Label'].to_numpy()



In [25]:
# Concatenate the two arrays into a single 2D numpy array
#X_train = np.concatenate((X_train, assay_id), axis=1)

In [26]:
# specify the parameters
#params = {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 5, 'n_estimators': 400}
# Train a Random Forest classifier on all the data
#clf = RandomForestClassifier(**params)
#clf.fit(X_train, y_train)


In [27]:
# Train a Random Forest classifier on all the data
#clf = RandomForestClassifier(**params)
#clf.fit(X_train, y_train)


In [28]:
# Predict the labels for the training data
#y_pred = clf.predict(X_train)

# Evaluate the accuracy of the classifier
#accuracy = np.mean(y_pred == y_train)
#print("Accuracy: {:.2f}%".format(accuracy * 100))

In [29]:
test_data.head()

,x
0,CC1=CC(=C(C=C1)C(C)(C)C)O;1682
1,CCS(=O)(=O)C1=C(N=CC=C1)S(=O)(=O)NC(=O)NC2=NC(...
2,CC1=NC2=CC=CC=C2N1C3CC4CCC(C3)N4CCC5(CCN(CC5)C...
3,CC1=CC(=O)[N-]S(=O)(=O)O1.[K+];1850
4,CCC(C)(C)C(=O)O[C@H]1C[C@H](C=C2[C@H]1[C@H]([C...


In [30]:
test_data[['Chemical ID', 'Assay ID']] = test_data['x'].str.split(';', expand=True)

In [31]:
test_data.head()

,x,Chemical ID,Assay ID
0,CC1=CC(=C(C=C1)C(C)(C)C)O;1682,CC1=CC(=C(C=C1)C(C)(C)C)O,1682
1,CCS(=O)(=O)C1=C(N=CC=C1)S(=O)(=O)NC(=O)NC2=NC(...,CCS(=O)(=O)C1=C(N=CC=C1)S(=O)(=O)NC(=O)NC2=NC(...,1656
2,CC1=NC2=CC=CC=C2N1C3CC4CCC(C3)N4CCC5(CCN(CC5)C...,CC1=NC2=CC=CC=C2N1C3CC4CCC(C3)N4CCC5(CCN(CC5)C...,36
3,CC1=CC(=O)[N-]S(=O)(=O)O1.[K+];1850,CC1=CC(=O)[N-]S(=O)(=O)O1.[K+],1850
4,CCC(C)(C)C(=O)O[C@H]1C[C@H](C=C2[C@H]1[C@H]([C...,CCC(C)(C)C(=O)O[C@H]1C[C@H](C=C2[C@H]1[C@H]([C...,30


In [32]:
test_data = test_data.drop('x', axis=1)

In [33]:
test_data.head()

,Chemical ID,Assay ID
0,CC1=CC(=C(C=C1)C(C)(C)C)O,1682
1,CCS(=O)(=O)C1=C(N=CC=C1)S(=O)(=O)NC(=O)NC2=NC(...,1656
2,CC1=NC2=CC=CC=C2N1C3CC4CCC(C3)N4CCC5(CCN(CC5)C...,36
3,CC1=CC(=O)[N-]S(=O)(=O)O1.[K+],1850
4,CCC(C)(C)C(=O)O[C@H]1C[C@H](C=C2[C@H]1[C@H]([C...,30


In [34]:
# Create a list to store the SMILES strings that do not cause an error
valid_smiles = []

# Iterate over the "Chemical ID" column and store only the valid SMILES strings
for smiles in test_data["Chemical ID"]:
    try:
        mols = Chem.MolFromSmiles(smiles, sanitize=False)
        if mols is not None:
            valid_smiles.append(smiles)
    except:
        continue

# Filter the DataFrame to only include rows with valid SMILES strings
test_data = test_data[test_data["Chemical ID"].isin(valid_smiles)]

In [35]:
ted_chem_aid_df = test_data[['Chemical ID', 'Assay ID']]

In [36]:
test_data['mols'] = test_data['Chemical ID'].apply(lambda x: Chem.MolFromSmiles(x))

In [37]:
test_data.head()

,Chemical ID,Assay ID,mols
0,CC1=CC(=C(C=C1)C(C)(C)C)O,1682,<rdkit.Chem.rdchem.Mol object at 0x000002363E1...
1,CCS(=O)(=O)C1=C(N=CC=C1)S(=O)(=O)NC(=O)NC2=NC(...,1656,<rdkit.Chem.rdchem.Mol object at 0x000002363E1...
2,CC1=NC2=CC=CC=C2N1C3CC4CCC(C3)N4CCC5(CCN(CC5)C...,36,<rdkit.Chem.rdchem.Mol object at 0x000002363E1...
3,CC1=CC(=O)[N-]S(=O)(=O)O1.[K+],1850,<rdkit.Chem.rdchem.Mol object at 0x000002363E1...
4,CCC(C)(C)C(=O)O[C@H]1C[C@H](C=C2[C@H]1[C@H]([C...,30,<rdkit.Chem.rdchem.Mol object at 0x000002363E1...


In [38]:
# Generate the Morgan fingerprints from the RDkit objects
#test_data['morgan_fp'] = test_data['mols'].apply(lambda x: rdMolDescriptors.GetMorganFingerprintAsBitVect(x, radius =4 , nBits = 300, useFeatures=True, useChirality=True))


In [39]:
test_data.isnull().sum()

Chemical ID    0
Assay ID       0
mols           0
dtype: int64

In [40]:
desc_df.isnull().sum()

MaxEStateIndex       0
MinEStateIndex       0
MaxAbsEStateIndex    0
MinAbsEStateIndex    0
qed                  0
                    ..
fr_thiazole          0
fr_thiocyan          0
fr_thiophene         0
fr_unbrch_alkane     0
fr_urea              0
Length: 203, dtype: int64

In [41]:
desc_df.head()

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,9.316200,-1.533785,9.316200,0.150485,0.794714,317.599,306.511,315.982463,100,0,...,0,0,0,0,0,0,0,0,0,0
1,10.532611,0.333788,10.532611,0.333788,0.516641,156.269,136.109,156.151415,66,0,...,0,0,0,0,0,0,0,0,4,0
2,2.433032,0.000000,2.433032,0.000000,0.251327,362.086,313.702,361.347528,148,0,...,0,0,0,0,0,0,0,0,12,0
3,10.355080,-0.613825,10.355080,0.282361,0.487998,255.665,245.585,255.052302,90,0,...,0,0,0,0,0,0,0,0,0,0
4,0.000000,0.000000,0.000000,0.000000,0.237972,149.894,149.894,149.894242,8,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:

train_data.head()

,Label,Chemical ID,Assay ID,mols
0,2,C1=CC(=CC=C1C(C2=CC=C(C=C2)O)C(Cl)(Cl)Cl)O,1644,<rdkit.Chem.rdchem.Mol object at 0x000002363FC...
1,2,CCCCCCCCC(=O)C,2451,<rdkit.Chem.rdchem.Mol object at 0x000002363FC...
2,2,CCCCCCCCCC[N+](C)(C)CCCCCCCCCC.[Cl-],1384,<rdkit.Chem.rdchem.Mol object at 0x000002363FC...
3,2,C1CN(C(=N1)N[N+](=O)[O-])CC2=CN=C(C=C2)Cl,16,<rdkit.Chem.rdchem.Mol object at 0x000002363FC...
4,2,[Na+].[I-],1856,<rdkit.Chem.rdchem.Mol object at 0x000002363FC...


In [43]:
# Normalize the molecular descriptors using MinMaxScaler
#scaler = MinMaxScaler()
#norm_desc_df = pd.DataFrame(scaler.fit_transform(desc_df), columns=desc_df.columns)

In [44]:
# Combine the molecular descriptors with the label and assay ID columns
#feat_df = pd.concat([norm_desc_df, train_data['Assay ID']])


In [45]:
# Generate additional molecular descriptors using RDKit
#train_morgan = np.array(train_data['morgan_fp'].tolist())

# Normalize the fingerprints using StandardScaler
#scaler = StandardScaler()
#X_morgan_scaled = scaler.fit_transform(train_morgan)
#train_data['mol_weight'] = train_data['mols'].apply(Chem.Descriptors.ExactMolWt) 
#train_data['logP'] = train_data['mols'].apply(Chem.Crippen.MolLogP)
#train_data['H_bond_donor_count'] = train_data['mols'].apply(Chem.Descriptors.NumHDonors)
#train_data['H_bond_acceptor_count'] = train_data['mols'].apply(Chem.Descriptors.NumHAcceptors)
#train_data['TPSA'] = train_data['mols'].apply(lambda m: rdMolDescriptors.CalcTPSA(m))

In [46]:
desc_df.head()

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,9.316200,-1.533785,9.316200,0.150485,0.794714,317.599,306.511,315.982463,100,0,...,0,0,0,0,0,0,0,0,0,0
1,10.532611,0.333788,10.532611,0.333788,0.516641,156.269,136.109,156.151415,66,0,...,0,0,0,0,0,0,0,0,4,0
2,2.433032,0.000000,2.433032,0.000000,0.251327,362.086,313.702,361.347528,148,0,...,0,0,0,0,0,0,0,0,12,0
3,10.355080,-0.613825,10.355080,0.282361,0.487998,255.665,245.585,255.052302,90,0,...,0,0,0,0,0,0,0,0,0,0
4,0.000000,0.000000,0.000000,0.000000,0.237972,149.894,149.894,149.894242,8,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
desc_df.isnull().sum()

MaxEStateIndex       0
MinEStateIndex       0
MaxAbsEStateIndex    0
MinAbsEStateIndex    0
qed                  0
                    ..
fr_thiazole          0
fr_thiocyan          0
fr_thiophene         0
fr_unbrch_alkane     0
fr_urea              0
Length: 203, dtype: int64

In [48]:
desc_df.columns = desc_df.columns.astype(str)

# Replace NaN and infinity values with the mean of the column
desc_df.replace([np.inf, -np.inf], np.nan, inplace=True)
# Select only the numeric columns
numeric_cols = desc_df.select_dtypes(include=np.number).columns

# Apply fillna() to the selected numeric columns
desc_df[numeric_cols] = desc_df[numeric_cols].fillna(desc_df[numeric_cols].median())

# Scale the input data using the StandardScaler
#scaler = StandardScaler()
#scaled_data = scaler.fit_transform(norm_desc_df)

# Convert the scaled data back to a pandas DataFrame
#scaled_df = pd.DataFrame(scaled_data, columns=norm_desc_df.columns)

In [49]:
# Get the fingerprints, assay ID, and labels from the dataframe
#X_train = np.array(train_data['mols'].values).reshape(-1, 1)
train_assay_id = np.array(train_data['Assay ID'].values).reshape(-1, 1)
#mol_weight = np.array(train_data['mol_weight'].values).reshape(-1, 1)
#logP = np.array(train_data['logP'].values).reshape(-1, 1)
#H_bond_donor_count = np.array(train_data['H_bond_donor_count'].values).reshape(-1, 1)
#H_bond_acceptor_count = np.array(train_data['H_bond_acceptor_count'].values).reshape(-1, 1)
#TPSA = np.array(train_data['TPSA'].values).reshape(-1, 1)
y_tr = train_data['Label'].to_numpy()

In [50]:
# Combine the normalized molecular descriptors with the label and assay ID columns
#feat_df = pd.concat([norm_desc_df, train_data[['Assay ID', 'Label', 'mols']]], axis=1)


In [51]:
# Set the threshold for the variance threshold selector
#thresh = 0.05

# Create the variance threshold selector
#selector = VarianceThreshold(threshold=thresh)


In [52]:
# Fit the selector to the feature matrix and transform the matrix
#feat_array = selector.fit_transform(feat_df.drop(['Assay ID', 'Label', 'mols'], axis=1))


In [53]:
# Get the column names of the selected features
# Get the column names of the selected features
#feat_names = feat_df.drop(['Assay ID', 'Label', 'mols'], axis=1).columns[selector.get_support()]


In [54]:
#desc_df = desc_df.drop(['fr_lactam', 'fr_dihydropyridine'], axis=1)

In [55]:
desc_df.head()

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,9.316200,-1.533785,9.316200,0.150485,0.794714,317.599,306.511,315.982463,100,0,...,0,0,0,0,0,0,0,0,0,0
1,10.532611,0.333788,10.532611,0.333788,0.516641,156.269,136.109,156.151415,66,0,...,0,0,0,0,0,0,0,0,4,0
2,2.433032,0.000000,2.433032,0.000000,0.251327,362.086,313.702,361.347528,148,0,...,0,0,0,0,0,0,0,0,12,0
3,10.355080,-0.613825,10.355080,0.282361,0.487998,255.665,245.585,255.052302,90,0,...,0,0,0,0,0,0,0,0,0,0
4,0.000000,0.000000,0.000000,0.000000,0.237972,149.894,149.894,149.894242,8,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
desc_df.isnull().sum()

MaxEStateIndex       0
MinEStateIndex       0
MaxAbsEStateIndex    0
MinAbsEStateIndex    0
qed                  0
                    ..
fr_thiazole          0
fr_thiocyan          0
fr_thiophene         0
fr_unbrch_alkane     0
fr_urea              0
Length: 203, dtype: int64

In [57]:
# Create a new DataFrame with the selected features and label and assay ID columns
#selected_feats = pd.DataFrame(data=feat_array, columns=feat_names)
#selected_feats = pd.concat([selected_feats, train_data[['Assay ID', 'Label', 'mols']]], axis=1)

# Split the selected features into X (features) and y (labels)
#X_train = feat_df.iloc[:, :-1]
#X_train = norm_desc_df.iloc[:, :-1]
#X_train = np.concatenate((trainassay_id, desc_df.iloc[:, :-1]), axis=1)
y_tr = train_data['Label'].to_numpy()

In [58]:
test_data.head()

,Chemical ID,Assay ID,mols
0,CC1=CC(=C(C=C1)C(C)(C)C)O,1682,<rdkit.Chem.rdchem.Mol object at 0x000002363E1...
1,CCS(=O)(=O)C1=C(N=CC=C1)S(=O)(=O)NC(=O)NC2=NC(...,1656,<rdkit.Chem.rdchem.Mol object at 0x000002363E1...
2,CC1=NC2=CC=CC=C2N1C3CC4CCC(C3)N4CCC5(CCN(CC5)C...,36,<rdkit.Chem.rdchem.Mol object at 0x000002363E1...
3,CC1=CC(=O)[N-]S(=O)(=O)O1.[K+],1850,<rdkit.Chem.rdchem.Mol object at 0x000002363E1...
4,CCC(C)(C)C(=O)O[C@H]1C[C@H](C=C2[C@H]1[C@H]([C...,30,<rdkit.Chem.rdchem.Mol object at 0x000002363E1...


In [59]:
# Calculate the molecular descriptors for each molecule
descs = []
for i, mol in enumerate(test_data['mols']):
    desc = []
    for d in Descriptors.descList:
        try:
            desc.append(d[1](mol))
        except:
            desc.append(np.nan)
    descs.append(desc)

In [60]:
# Create a DataFrame of the molecular descriptors
test_desc_df = pd.DataFrame(descs, columns=[d[0] for d in Descriptors.descList])

In [61]:
all_zero_cols = [col for col in test_desc_df.columns if test_desc_df[col].sum() == 0]
test_desc_df = test_desc_df.drop(all_zero_cols, axis=1)


In [62]:
test_desc_df.to_csv('testdescriptors.csv', index=False)

In [63]:
# Set threshold for minimum variance
#threshold = 0.5

# Create variance threshold object and fit to data
#sel = VarianceThreshold(threshold)
#sel.fit(test_desc_df)

# Remove low-variance features from data
#test_desc_df_new = sel.transform(test_desc_df)

# Convert numpy ndarray to pandas DataFrame
#new_df_test = pd.DataFrame(test_desc_df_new, columns=test_desc_df.columns[sel.get_support()])

In [64]:
test_desc_df.head()

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,9.626968,0.025579,9.626968,0.025579,0.624614,164.248,148.120,164.120115,66,0,...,0,0,0,0,0,0,0,0,0,0
1,12.473362,-4.605249,12.473362,0.061165,0.617511,431.452,414.316,431.056940,152,0,...,0,1,1,0,0,0,0,0,0,1
2,14.627193,-4.140552,14.627193,0.064351,0.224134,696.264,655.944,695.250845,254,0,...,0,1,0,0,0,0,0,0,0,0
3,10.420833,-3.973958,10.420833,0.000000,0.393203,201.244,197.212,200.949810,56,0,...,0,0,0,0,0,0,0,0,0,0
4,12.865865,-0.601027,12.865865,0.094949,0.639062,418.574,380.270,418.271924,168,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
len(test_desc_df)

10994

In [66]:
test_desc_df.columns = test_desc_df.columns.astype(str)

# Replace NaN and infinity values with the mean of the column
test_desc_df.replace([np.inf, -np.inf], np.nan, inplace=True)
# Select only the numeric columns
numeric_cols = test_desc_df.select_dtypes(include=np.number).columns

# Apply fillna() to the selected numeric columns
test_desc_df[numeric_cols] = test_desc_df[numeric_cols].fillna(test_desc_df[numeric_cols].median())

# Scale the input data using the StandardScaler
#scaler = StandardScaler()
#scaled_data = scaler.fit_transform(test_norm_desc_df)

# Convert the scaled data back to a pandas DataFrame
#test_scaled_df = pd.DataFrame(scaled_data, columns=test_norm_desc_df.columns)

In [67]:
# get the column names of each data frame
cols1 = set(desc_df.columns)
cols2 = set(test_desc_df.columns)

# find the columns that are unique to each data frame
uncommon_cols1 = cols1 - cols2
uncommon_cols2 = cols2 - cols1

# print the uncommon columns
print(f'Columns unique to dataset 1: {uncommon_cols1}')
print(f'Columns unique to dataset 2: {uncommon_cols2}')

Columns unique to dataset 1: {'fr_lactam', 'fr_dihydropyridine'}
Columns unique to dataset 2: set()


In [68]:
#assay_id = np.array(test_data['Assay ID'].values).reshape(-1, 1)

#X_test = np.concatenate((X_test, assay_id), axis=1)

#X_test = np.array(test_data['mols'].values).reshape(-1, 1)
#test_morgan = np.array(test_data['morgan_fp'].tolist())
# Normalize the fingerprints using StandardScaler
#scaler = StandardScaler()
#X_morgan_scaled_test = scaler.fit_transform(test_morgan)
test_assay_id = np.array(test_data['Assay ID'].values).reshape(-1, 1)
#mol_weight = np.array(test_data['mol_weight'].values).reshape(-1, 1)
#logP = np.array(test_data['logP'].values).reshape(-1, 1)
#H_bond_donor_count = np.array(test_data['H_bond_donor_count'].values).reshape(-1, 1)
#H_bond_acceptor_count = np.array(test_data['H_bond_acceptor_count'].values).reshape(-1, 1)
#TPSA = np.array(test_data['TPSA'].values).reshape(-1, 1)

In [69]:
# Set up the PCA model
n_components = 25 # Choose the number of principal components to keep
pca = PCA(n_components)

# Fit the PCA model to the data
pca.fit(desc_df)

# Get the explained variance ratio of each principal component
variance_ratio = pca.explained_variance_ratio_

# Print the variance explained by each principal component
#print("Variance explained by each principal component:", variance_ratio)

# Get the loadings of each descriptor onto each principal component
loadings = pd.DataFrame(pca.components_.T, columns=[f"PC{i+1}" for i in range(n_components)], index=desc_df.columns)

# Print the loadings for the first principal component
#print("Loadings for PC1:", loadings["PC1"])

# Select the top k descriptors based on their absolute loading values in PC1
k = 105  # Choose the number of top descriptors to select
top_50_descriptors_with_PCA = loadings["PC1"].abs().sort_values(ascending=False)[:k].index

# Print the top descriptors
#print("Top descriptors:", top_50_descriptors_with_PCA)
top_50_descriptors_with_PCA = ", ".join([f"'{name}'" for name in top_50_descriptors_with_PCA])
print(top_50_descriptors_with_PCA)

'Ipc', 'BertzCT', 'MolWt', 'ExactMolWt', 'HeavyAtomMolWt', 'TPSA', 'LabuteASA', 'NumValenceElectrons', 'EState_VSA1', 'MolMR', 'VSA_EState3', 'SMR_VSA1', 'SlogP_VSA2', 'PEOE_VSA1', 'SlogP_VSA11', 'SMR_VSA9', 'PEOE_VSA11', 'EState_VSA10', 'SMR_VSA7', 'VSA_EState2', 'EState_VSA3', 'HeavyAtomCount', 'SlogP_VSA6', 'PEOE_VSA7', 'SlogP_VSA5', 'Chi0', 'Kappa1', 'VSA_EState5', 'PEOE_VSA14', 'Chi0n', 'Chi1', 'Chi0v', 'VSA_EState1', 'PEOE_VSA9', 'PEOE_VSA3', 'NumHAcceptors', 'NOCount', 'NumHeteroatoms', 'SMR_VSA10', 'EState_VSA9', 'PEOE_VSA12', 'Chi1n', 'Kappa2', 'Chi1v', 'PEOE_VSA6', 'fr_phenol_noOrthoHbond', 'fr_phenol', 'fr_Ar_OH', 'NumHDonors', 'NHOHCount', 'Chi2n', 'Chi2v', 'SlogP_VSA3', 'SlogP_VSA1', 'NumRotatableBonds', 'HallKierAlpha', 'VSA_EState7', 'EState_VSA11', 'Chi3n', 'Chi3v', 'EState_VSA8', 'EState_VSA7', 'EState_VSA2', 'EState_VSA5', 'EState_VSA4', 'PEOE_VSA8', 'VSA_EState4', 'Kappa3', 'fr_ether', 'BCUT2D_MWHI', 'EState_VSA6', 'SlogP_VSA12', 'fr_ester', 'fr_C_O_noCOO', 'RingCoun

In [70]:
#rf = RandomForestRegressor(n_estimators=500)
#clf =  RandomForestClassifier(n_estimators= 700)
xgb = XGBClassifier(n_estimators=1050)

# define Gradient Boosted Trees model
#gbt = GradientBoostingClassifier(learning_rate=0.1, max_depth=3, n_estimators=500, random_state=42)



In [71]:
X_tr = np.concatenate((train_assay_id, desc_df[['Ipc', 'BertzCT', 'MolWt', 'ExactMolWt', 'HeavyAtomMolWt', 'TPSA', 'LabuteASA', 'NumValenceElectrons', 'EState_VSA1', 'MolMR', 'VSA_EState3', 'SMR_VSA1', 'SlogP_VSA2', 'PEOE_VSA1', 'SlogP_VSA11', 'SMR_VSA9', 'PEOE_VSA11', 'EState_VSA10', 'SMR_VSA7', 'VSA_EState2', 'EState_VSA3', 'HeavyAtomCount', 'SlogP_VSA6', 'PEOE_VSA7', 'SlogP_VSA5', 'Chi0', 'Kappa1', 'VSA_EState5', 'PEOE_VSA14', 'Chi0n', 'Chi1', 'Chi0v', 'VSA_EState1', 'PEOE_VSA9', 'PEOE_VSA3', 'NumHAcceptors', 'NOCount', 'NumHeteroatoms', 'SMR_VSA10', 'EState_VSA9', 'PEOE_VSA12', 'Chi1n', 'Kappa2', 'Chi1v', 'PEOE_VSA6', 'fr_phenol_noOrthoHbond', 'fr_phenol', 'fr_Ar_OH', 'NumHDonors', 'NHOHCount', 'Chi2n', 'Chi2v', 'SlogP_VSA3', 'SlogP_VSA1', 'NumRotatableBonds', 'HallKierAlpha', 'VSA_EState7', 'EState_VSA11', 'Chi3n', 'Chi3v', 'EState_VSA8', 'EState_VSA7', 'EState_VSA2', 'EState_VSA5', 'EState_VSA4', 'PEOE_VSA8', 'VSA_EState4', 'Kappa3', 'fr_ether', 'BCUT2D_MWHI', 'EState_VSA6', 'SlogP_VSA12', 'fr_ester', 'fr_C_O_noCOO', 'RingCount', 'fr_C_O', 'fr_benzene', 'NumAromaticCarbocycles', 'Chi4n', 'NumAromaticRings', 'Chi4v', 'PEOE_VSA10', 'MaxAbsEStateIndex', 'SMR_VSA5', 'SlogP_VSA10', 'SlogP_VSA4', 'VSA_EState8', 'SMR_VSA3', 'PEOE_VSA2', 'VSA_EState10', 'SMR_VSA6', 'PEOE_VSA4', 'VSA_EState6', 'SMR_VSA4', 'MolLogP', 'SlogP_VSA8', 'PEOE_VSA5', 'MinEStateIndex', 'SlogP_VSA7', 'PEOE_VSA13', 'FpDensityMorgan3', 'BalabanJ', 'FpDensityMorgan2', 'fr_halogen', 'fr_NH0']]), axis=1) 
X_te = np.concatenate((test_assay_id, test_desc_df[['Ipc', 'BertzCT', 'MolWt', 'ExactMolWt', 'HeavyAtomMolWt', 'TPSA', 'LabuteASA', 'NumValenceElectrons', 'EState_VSA1', 'MolMR', 'VSA_EState3', 'SMR_VSA1', 'SlogP_VSA2', 'PEOE_VSA1', 'SlogP_VSA11', 'SMR_VSA9', 'PEOE_VSA11', 'EState_VSA10', 'SMR_VSA7', 'VSA_EState2', 'EState_VSA3', 'HeavyAtomCount', 'SlogP_VSA6', 'PEOE_VSA7', 'SlogP_VSA5', 'Chi0', 'Kappa1', 'VSA_EState5', 'PEOE_VSA14', 'Chi0n', 'Chi1', 'Chi0v', 'VSA_EState1', 'PEOE_VSA9', 'PEOE_VSA3', 'NumHAcceptors', 'NOCount', 'NumHeteroatoms', 'SMR_VSA10', 'EState_VSA9', 'PEOE_VSA12', 'Chi1n', 'Kappa2', 'Chi1v', 'PEOE_VSA6', 'fr_phenol_noOrthoHbond', 'fr_phenol', 'fr_Ar_OH', 'NumHDonors', 'NHOHCount', 'Chi2n', 'Chi2v', 'SlogP_VSA3', 'SlogP_VSA1', 'NumRotatableBonds', 'HallKierAlpha', 'VSA_EState7', 'EState_VSA11', 'Chi3n', 'Chi3v', 'EState_VSA8', 'EState_VSA7', 'EState_VSA2', 'EState_VSA5', 'EState_VSA4', 'PEOE_VSA8', 'VSA_EState4', 'Kappa3', 'fr_ether', 'BCUT2D_MWHI', 'EState_VSA6', 'SlogP_VSA12', 'fr_ester', 'fr_C_O_noCOO', 'RingCount', 'fr_C_O', 'fr_benzene', 'NumAromaticCarbocycles', 'Chi4n', 'NumAromaticRings', 'Chi4v', 'PEOE_VSA10', 'MaxAbsEStateIndex', 'SMR_VSA5', 'SlogP_VSA10', 'SlogP_VSA4', 'VSA_EState8', 'SMR_VSA3', 'PEOE_VSA2', 'VSA_EState10', 'SMR_VSA6', 'PEOE_VSA4', 'VSA_EState6', 'SMR_VSA4', 'MolLogP', 'SlogP_VSA8', 'PEOE_VSA5', 'MinEStateIndex', 'SlogP_VSA7', 'PEOE_VSA13', 'FpDensityMorgan3', 'BalabanJ', 'FpDensityMorgan2', 'fr_halogen', 'fr_NH0']]), axis=1)

X_tr_ALL = np.concatenate((train_assay_id, desc_df), axis=1) 
X_te_ALL = np.concatenate((test_assay_id, test_desc_df), axis=1)


In [72]:
#smote = SMOTE()

#X_train_balanced, y_train_balanced = smote.fit_resample(X_tr, y_tr)

In [73]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_tr, y_tr, test_size=0.2, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(X_train_balanced, y_train_balanced, test_size=0.2, random_state=42)

In [74]:
y_tr[y_tr == 2] = 0
y_tr[y_tr == 1] = 1

In [75]:
#y_train_balanced[y_train_balanced == 2] = 0
#y_train_balanced[y_train_balanced == 1] = 1


In [76]:
#the F1 Score
xgb.fit(X_tr, y_tr)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=1050, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [77]:
# Predict labels on the test set
y_pred = xgb.predict(X_test)

# Calculate F1 score for the test set
f1= f1_score(y_test, y_pred, average='weighted')
print("F1 Score:", f1)

F1 Score: 0.13941461560105547


In [78]:
Z_pred = xgb.predict(X_te)

In [79]:
print(Z_pred)

[0 0 0 ... 0 0 1]


In [80]:
Z_pred[Z_pred == 0] = 2
Z_pred[Z_pred == 1] = 1

In [81]:
# Combine the SMILES & Assay ID into a single column
test_data['Id'] = test_data['Chemical ID'] + ';' + test_data['Assay ID'].astype(str)

# Create a new dataframe with the Id and Predicted columns
result_data = pd.DataFrame({'Id': test_data['Id'], 'Predicted': Z_pred})

# Save the result data to a csv file
result_data.to_csv('110+1350.csv', index=False)